# Router 주문 응대 Classify Agent - 커피 키오스크 주문 시스템

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [ ]:
# 필요한 라이브러리 설치
# !pip install openai pydantic

In [2]:
import os
import json
from typing import List, Dict, Optional, Literal
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel, Field, validator
from enum import Enum

# OpenAI 클라이언트 초기화
client = OpenAI()

## 2. Pydantic 모델 정의

In [3]:
class MenuCategory(str, Enum):
    """메뉴 카테고리 열거형"""
    COFFEE = "coffee"
    TEA = "tea"
    DESSERT = "dessert"
    SIDE = "side"
    UNKNOWN = "unknown"


class Temperature(str, Enum):
    """음료 온도 열거형"""
    HOT = "hot"
    ICED = "iced"
    NONE = "none"  # 온도가 적용되지 않는 항목


class OrderItem(BaseModel):
    """개별 주문 항목을 표현하는 모델"""
    item_name: str = Field(description="주문 항목 이름")
    category: MenuCategory = Field(description="메뉴 카테고리")
    quantity: int = Field(ge=1, le=10, description="수량 (1-10)")
    temperature: Temperature = Field(default=Temperature.NONE, description="온도 옵션")
    size: Optional[str] = Field(default="regular", description="사이즈 (small/regular/large)")
    options: List[str] = Field(default_factory=list, description="추가 옵션 (샷 추가, 시럽 등)")
    price: float = Field(ge=0, description="가격")
    
    @validator('item_name')
    def validate_item_name(cls, v):
        """항목 이름이 비어있지 않은지 검증한다"""
        if not v or not v.strip():
            raise ValueError("항목 이름은 비어있을 수 없다")
        return v.strip()


class ParsedOrder(BaseModel):
    """파싱된 전체 주문을 표현하는 모델"""
    items: List[OrderItem] = Field(description="주문 항목 리스트")
    customer_notes: Optional[str] = Field(default=None, description="고객 요청 사항")
    original_text: str = Field(description="원본 주문 텍스트")
    total_items: int = Field(ge=0, description="총 항목 수")
    
    @validator('total_items', always=True)
    def calculate_total_items(cls, v, values):
        """총 항목 수를 계산한다"""
        if 'items' in values:
            return sum(item.quantity for item in values['items'])
        return v


class CategoryClassification(BaseModel):
    """카테고리 분류 결과를 표현하는 모델"""
    category: MenuCategory = Field(description="분류된 카테고리")
    items: List[OrderItem] = Field(description="해당 카테고리의 항목들")
    requires_special_handling: bool = Field(
        default=False,
        description="특별 처리가 필요한지 여부"
    )
    handling_notes: Optional[str] = Field(
        default=None,
        description="특별 처리 사항"
    )


class OrderValidation(BaseModel):
    """주문 검증 결과를 표현하는 모델"""
    is_valid: bool = Field(description="주문의 유효성")
    invalid_items: List[str] = Field(default_factory=list, description="유효하지 않은 항목들")
    validation_errors: List[str] = Field(default_factory=list, description="검증 오류 메시지들")
    warnings: List[str] = Field(default_factory=list, description="경고 메시지들")
    total_price: float = Field(ge=0, description="총 가격")


class ConversationMessage(BaseModel):
    """대화 메시지를 표현하는 모델"""
    role: Literal["user", "assistant", "system"] = Field(description="메시지 역할")
    content: str = Field(description="메시지 내용")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    order_data: Optional[ParsedOrder] = Field(default=None, description="주문 데이터")


class AgentResponse(BaseModel):
    """에이전트의 최종 응답을 표현하는 모델"""
    message: str = Field(description="고객에게 제공되는 메시지")
    parsed_order: ParsedOrder = Field(description="파싱된 주문")
    classifications: List[CategoryClassification] = Field(description="카테고리별 분류")
    validation: OrderValidation = Field(description="주문 검증 결과")
    suggested_additions: List[str] = Field(default_factory=list, description="추가 제안 항목들")
    requires_confirmation: bool = Field(description="고객 확인이 필요한지 여부")

/var/folders/v9/46y9d8bn1lxgjt7g439hsf8c0000gn/T/ipykernel_92818/3045768978.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('item_name')
/var/folders/v9/46y9d8bn1lxgjt7g439hsf8c0000gn/T/ipykernel_92818/3045768978.py:42: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('total_items', always=True)


## 3. 메뉴 데이터베이스 구축

In [4]:
# 메뉴 데이터베이스
MENU_DATABASE = {
    MenuCategory.COFFEE: {
        "아메리카노": {"base_price": 4500, "sizes": ["regular", "large"], "temperature": True},
        "카페라떼": {"base_price": 5000, "sizes": ["regular", "large"], "temperature": True},
        "카푸치노": {"base_price": 5000, "sizes": ["regular", "large"], "temperature": True},
        "바닐라라떼": {"base_price": 5500, "sizes": ["regular", "large"], "temperature": True},
        "카라멜마끼아또": {"base_price": 5800, "sizes": ["regular", "large"], "temperature": True},
        "에스프레소": {"base_price": 3500, "sizes": ["single", "double"], "temperature": False},
    },
    MenuCategory.TEA: {
        "녹차": {"base_price": 4500, "sizes": ["regular", "large"], "temperature": True},
        "홍차": {"base_price": 4500, "sizes": ["regular", "large"], "temperature": True},
        "녹차라떼": {"base_price": 5500, "sizes": ["regular", "large"], "temperature": True},
        "얼그레이": {"base_price": 4800, "sizes": ["regular", "large"], "temperature": True},
        "캐모마일": {"base_price": 4800, "sizes": ["regular", "large"], "temperature": True},
    },
    MenuCategory.DESSERT: {
        "치즈케이크": {"base_price": 6500, "sizes": ["1piece"], "temperature": False},
        "초코케이크": {"base_price": 6500, "sizes": ["1piece"], "temperature": False},
        "마카롱": {"base_price": 2500, "sizes": ["1piece"], "temperature": False},
        "쿠키": {"base_price": 3000, "sizes": ["1piece"], "temperature": False},
        "티라미수": {"base_price": 7000, "sizes": ["1piece"], "temperature": False},
        "스콘": {"base_price": 3500, "sizes": ["1piece"], "temperature": False},
    },
    MenuCategory.SIDE: {
        "햄치즈샌드위치": {"base_price": 5500, "sizes": ["regular"], "temperature": False},
        "베이글": {"base_price": 4500, "sizes": ["regular"], "temperature": False},
        "크루아상": {"base_price": 4000, "sizes": ["regular"], "temperature": False},
        "샐러드": {"base_price": 8500, "sizes": ["regular"], "temperature": False},
    }
}

# 재고 정보 (시뮬레이션)
INVENTORY = {
    "아메리카노": 100,
    "카페라떼": 100,
    "카푸치노": 80,
    "바닐라라떼": 50,
    "녹차": 60,
    "홍차": 60,
    "치즈케이크": 10,
    "초코케이크": 8,
    "마카롱": 30,
    "햄치즈샌드위치": 15,
}

# 추가 옵션 가격
OPTION_PRICES = {
    "샷추가": 500,
    "시럽추가": 500,
    "휘핑추가": 700,
    "두유변경": 600,
    "저지방우유": 300,
}

# 사이즈 추가 가격
SIZE_PRICES = {
    "regular": 0,
    "large": 500,
    "small": -500,
    "single": 0,
    "double": 1000,
    "1piece": 0,
}

print(f"총 {sum(len(items) for items in MENU_DATABASE.values())}개의 메뉴가 등록되었다.")
for category, items in MENU_DATABASE.items():
    print(f"  - {category.value}: {len(items)}개 항목")

총 21개의 메뉴가 등록되었다.
  - coffee: 6개 항목
  - tea: 5개 항목
  - dessert: 6개 항목
  - side: 4개 항목


## 4. Memory 시스템 구현

In [5]:
class OrderMemory:
    """주문 이력과 고객 선호도를 관리하는 메모리 클래스"""
    
    def __init__(self, max_history: int = 10):
        self.messages: List[ConversationMessage] = []
        self.max_history = max_history
        self.order_history: List[ParsedOrder] = []
        self.customer_preferences: Dict = {
            "favorite_items": {},
            "preferred_temperature": {},
            "preferred_size": {},
            "common_options": []
        }
        self.category_stats: Dict[str, int] = {
            "coffee": 0,
            "tea": 0,
            "dessert": 0,
            "side": 0
        }
    
    def add_message(self, role: str, content: str, order_data: ParsedOrder = None):
        """새로운 메시지를 메모리에 추가한다"""
        message = ConversationMessage(
            role=role,
            content=content,
            order_data=order_data
        )
        self.messages.append(message)
        
        # 주문 데이터가 있으면 이력에 추가
        if order_data:
            self.order_history.append(order_data)
            self._update_preferences(order_data)
        
        # 최대 이력 수를 초과하면 오래된 메시지부터 제거한다
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
    
    def _update_preferences(self, order: ParsedOrder):
        """주문 데이터를 바탕으로 고객 선호도를 업데이트한다"""
        for item in order.items:
            # 선호 항목 카운트
            if item.item_name not in self.customer_preferences["favorite_items"]:
                self.customer_preferences["favorite_items"][item.item_name] = 0
            self.customer_preferences["favorite_items"][item.item_name] += 1
            
            # 선호 온도
            if item.temperature != Temperature.NONE:
                temp_key = item.temperature.value
                if temp_key not in self.customer_preferences["preferred_temperature"]:
                    self.customer_preferences["preferred_temperature"][temp_key] = 0
                self.customer_preferences["preferred_temperature"][temp_key] += 1
            
            # 선호 사이즈
            if item.size:
                if item.size not in self.customer_preferences["preferred_size"]:
                    self.customer_preferences["preferred_size"][item.size] = 0
                self.customer_preferences["preferred_size"][item.size] += 1
            
            # 옵션 추적
            for option in item.options:
                if option not in self.customer_preferences["common_options"]:
                    self.customer_preferences["common_options"].append(option)
            
            # 카테고리 통계
            if item.category.value in self.category_stats:
                self.category_stats[item.category.value] += 1
    
    def get_favorite_items(self, top_n: int = 3) -> List[str]:
        """가장 자주 주문한 항목들을 반환한다"""
        favorites = self.customer_preferences["favorite_items"]
        sorted_items = sorted(favorites.items(), key=lambda x: x[1], reverse=True)
        return [item[0] for item in sorted_items[:top_n]]
    
    def get_preferred_category(self) -> Optional[str]:
        """가장 선호하는 카테고리를 반환한다"""
        if not self.category_stats:
            return None
        return max(self.category_stats.items(), key=lambda x: x[1])[0]
    
    def get_context(self) -> List[Dict]:
        """OpenAI API 형식으로 대화 이력을 반환한다"""
        context = []
        for msg in self.messages:
            context.append({
                "role": msg.role,
                "content": msg.content
            })
        return context
    
    def get_summary(self) -> str:
        """메모리 상태 요약을 반환한다"""
        total_orders = len(self.order_history)
        favorite = self.get_favorite_items(1)
        favorite_str = favorite[0] if favorite else "없음"
        return f"총 주문: {total_orders}회, 선호 항목: {favorite_str}"
    
    def clear(self):
        """모든 이력을 삭제한다"""
        self.messages = []
        self.order_history = []

# 메모리 인스턴스 생성
memory = OrderMemory(max_history=10)

# 시스템 메시지 추가
memory.add_message(
    "system",
    "당신은 커피 키오스크의 주문 접수 전문가다. 고객의 주문을 정확하게 파악하고 친절하게 안내하라."
)

print("주문 메모리가 초기화되었다.")

주문 메모리가 초기화되었다.


## 5. 주문 파싱 시스템 구현

In [6]:
class OrderParser:
    """주문 파싱 클래스"""
    
    def __init__(self):
        self.parser_model = "gpt-4o-mini"
    
    def parse_order(self, order_text: str, context: List[Dict] = None) -> ParsedOrder:
        """
        자연어 주문을 구조화된 데이터로 파싱한다
        
        Args:
            order_text: 고객의 주문 텍스트
            context: 대화 컨텍스트
        
        Returns:
            ParsedOrder 객체
        """
        # 메뉴 정보를 포함한 프롬프트 생성
        menu_info = self._build_menu_info()
        
        parsing_prompt = f"""
고객의 주문을 파싱하여 구조화된 데이터로 변환하라.

주문 텍스트: "{order_text}"

사용 가능한 메뉴:
{menu_info}

파싱 규칙:
1. 항목 이름은 정확한 메뉴 이름으로 매칭
2. 카테고리 자동 분류 (coffee/tea/dessert/side)
3. 수량이 명시되지 않으면 1개로 간주
4. 온도 옵션 추출 (hot/iced/none)
5. 사이즈 추출 (small/regular/large)
6. 추가 옵션 추출 (샷추가, 시럽추가 등)
7. 각 항목의 기본 가격 설정

JSON 형식으로 답변하라:
{{
  "items": [
    {{
      "item_name": "정확한 메뉴명",
      "category": "coffee|tea|dessert|side",
      "quantity": 숫자,
      "temperature": "hot|iced|none",
      "size": "small|regular|large",
      "options": ["옵션1", "옵션2"],
      "price": 가격
    }}
  ],
  "customer_notes": "특별 요청사항",
  "original_text": "{order_text}",
  "total_items": 총개수
}}
"""
        
        messages = context or []
        messages.append({"role": "user", "content": parsing_prompt})
        
        response = client.chat.completions.create(
            model=self.parser_model,
            messages=messages,
            temperature=0.1,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        
        # Pydantic 모델로 변환
        parsed_order = ParsedOrder(**result)
        
        return parsed_order
    
    def _build_menu_info(self) -> str:
        """메뉴 정보를 문자열로 구성한다"""
        menu_lines = []
        for category, items in MENU_DATABASE.items():
            menu_lines.append(f"\n[{category.value.upper()}]")
            for name, info in items.items():
                price = info["base_price"]
                menu_lines.append(f"  - {name}: {price}원")
        return "\n".join(menu_lines)

# 주문 파서 인스턴스 생성
order_parser = OrderParser()
print("주문 파싱 시스템이 초기화되었다.")

주문 파싱 시스템이 초기화되었다.


## 6. Function Calling & Tool 정의

In [7]:
# 카테고리별 처리 도구 정의
CATEGORY_HANDLER_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "handle_coffee_order",
            "description": "커피 주문을 처리한다. 온도, 사이즈, 샷 추가 등의 옵션을 확인한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "items": {
                        "type": "array",
                        "description": "커피 항목 리스트",
                        "items": {"type": "string"}
                    },
                    "special_instructions": {
                        "type": "string",
                        "description": "특별 지시사항"
                    }
                },
                "required": ["items"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handle_tea_order",
            "description": "티 주문을 처리한다. 온도와 티백/파우더 선호도를 확인한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "items": {
                        "type": "array",
                        "description": "티 항목 리스트",
                        "items": {"type": "string"}
                    },
                    "special_instructions": {
                        "type": "string",
                        "description": "특별 지시사항"
                    }
                },
                "required": ["items"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handle_dessert_order",
            "description": "디저트 주문을 처리한다. 포장 여부와 알레르기 정보를 확인한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "items": {
                        "type": "array",
                        "description": "디저트 항목 리스트",
                        "items": {"type": "string"}
                    },
                    "special_instructions": {
                        "type": "string",
                        "description": "특별 지시사항"
                    }
                },
                "required": ["items"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handle_side_order",
            "description": "사이드 메뉴 주문을 처리한다. 데우기 여부와 소스 선택을 확인한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "items": {
                        "type": "array",
                        "description": "사이드 항목 리스트",
                        "items": {"type": "string"}
                    },
                    "special_instructions": {
                        "type": "string",
                        "description": "특별 지시사항"
                    }
                },
                "required": ["items"]
            }
        }
    }
]

print(f"총 {len(CATEGORY_HANDLER_TOOLS)}개의 카테고리 처리 도구가 정의되었다.")

총 4개의 카테고리 처리 도구가 정의되었다.


## 7. 카테고리 분류 라우터 구현

In [8]:
class CategoryRouter:
    """카테고리별 라우팅 클래스"""
    
    def __init__(self):
        self.router_model = "gpt-4o-mini"
    
    def classify_and_route(self, parsed_order: ParsedOrder) -> List[CategoryClassification]:
        """
        주문을 카테고리별로 분류하고 라우팅한다
        
        Args:
            parsed_order: 파싱된 주문 데이터
        
        Returns:
            CategoryClassification 리스트
        """
        # 카테고리별로 항목 그룹화
        category_groups: Dict[MenuCategory, List[OrderItem]] = {}
        
        for item in parsed_order.items:
            if item.category not in category_groups:
                category_groups[item.category] = []
            category_groups[item.category].append(item)
        
        # 각 카테고리별로 처리
        classifications = []
        
        for category, items in category_groups.items():
            # 특별 처리 필요 여부 판단
            requires_special, notes = self._check_special_handling(category, items)
            
            classification = CategoryClassification(
                category=category,
                items=items,
                requires_special_handling=requires_special,
                handling_notes=notes
            )
            
            classifications.append(classification)
            
            print(f"\n카테고리: {category.value}")
            print(f"  - 항목 수: {len(items)}개")
            print(f"  - 특별 처리: {'필요' if requires_special else '불필요'}")
            if notes:
                print(f"  - 처리 사항: {notes}")
        
        return classifications
    
    def _check_special_handling(self, category: MenuCategory, items: List[OrderItem]) -> tuple[bool, Optional[str]]:
        """
        카테고리와 항목을 분석하여 특별 처리가 필요한지 확인한다
        
        Returns:
            (특별 처리 필요 여부, 처리 사항)
        """
        notes = []
        
        # 대량 주문 체크
        total_quantity = sum(item.quantity for item in items)
        if total_quantity > 5:
            notes.append(f"대량 주문 ({total_quantity}개)")
        
        # 카테고리별 특별 조건 체크
        if category == MenuCategory.COFFEE:
            # 복잡한 옵션이 있는지 확인
            for item in items:
                if len(item.options) > 2:
                    notes.append(f"{item.item_name}에 복잡한 커스터마이징")
        
        elif category == MenuCategory.DESSERT:
            # 케이크류는 포장 확인 필요
            cake_items = [item for item in items if "케이크" in item.item_name]
            if cake_items:
                notes.append("케이크 포장 확인 필요")
        
        elif category == MenuCategory.SIDE:
            # 샌드위치는 데우기 확인 필요
            sandwich_items = [item for item in items if "샌드위치" in item.item_name]
            if sandwich_items:
                notes.append("샌드위치 데우기 확인 필요")
        
        requires_special = len(notes) > 0
        notes_str = ", ".join(notes) if notes else None
        
        return requires_special, notes_str

# 카테고리 라우터 인스턴스 생성
category_router = CategoryRouter()
print("카테고리 라우터가 초기화되었다.")

카테고리 라우터가 초기화되었다.


## 8. Validation 시스템 구현

In [9]:
class OrderValidationSystem:
    """주문 검증 시스템 클래스"""
    
    def __init__(self):
        self.validator_model = "gpt-4o-mini"
    
    def validate_order(self, parsed_order: ParsedOrder) -> OrderValidation:
        """
        주문의 유효성을 종합적으로 검증한다
        
        Args:
            parsed_order: 파싱된 주문 데이터
        
        Returns:
            OrderValidation 객체
        """
        invalid_items = []
        validation_errors = []
        warnings = []
        total_price = 0.0
        
        # 1. 메뉴 존재 여부 확인
        for item in parsed_order.items:
            if not self._is_valid_menu_item(item):
                invalid_items.append(item.item_name)
                validation_errors.append(f"{item.item_name}은(는) 메뉴에 없다")
        
        # 2. 재고 확인
        stock_errors = self._check_inventory(parsed_order.items)
        validation_errors.extend(stock_errors)
        
        # 3. 옵션 유효성 확인
        option_errors = self._validate_options(parsed_order.items)
        validation_errors.extend(option_errors)
        
        # 4. 가격 계산
        total_price = self._calculate_total_price(parsed_order.items)
        
        # 5. 조합 경고 확인
        combo_warnings = self._check_combinations(parsed_order.items)
        warnings.extend(combo_warnings)
        
        # 6. 수량 경고
        if parsed_order.total_items > 10:
            warnings.append("대량 주문입니다. 준비 시간이 더 걸릴 수 있다")
        
        is_valid = len(validation_errors) == 0
        
        return OrderValidation(
            is_valid=is_valid,
            invalid_items=invalid_items,
            validation_errors=validation_errors,
            warnings=warnings,
            total_price=total_price
        )
    
    def _is_valid_menu_item(self, item: OrderItem) -> bool:
        """메뉴에 존재하는 항목인지 확인한다"""
        if item.category == MenuCategory.UNKNOWN:
            return False
        
        category_menu = MENU_DATABASE.get(item.category, {})
        return item.item_name in category_menu
    
    def _check_inventory(self, items: List[OrderItem]) -> List[str]:
        """재고를 확인한다"""
        errors = []
        for item in items:
            if item.item_name in INVENTORY:
                available = INVENTORY[item.item_name]
                if available < item.quantity:
                    errors.append(
                        f"{item.item_name}의 재고가 부족하다 (요청: {item.quantity}, 재고: {available})"
                    )
        return errors
    
    def _validate_options(self, items: List[OrderItem]) -> List[str]:
        """옵션의 유효성을 검증한다"""
        errors = []
        for item in items:
            # 온도 옵션 확인
            if item.category in [MenuCategory.COFFEE, MenuCategory.TEA]:
                if item.temperature == Temperature.NONE:
                    errors.append(f"{item.item_name}의 온도 옵션이 지정되지 않았다")
            
            # 사이즈 확인
            category_menu = MENU_DATABASE.get(item.category, {})
            if item.item_name in category_menu:
                valid_sizes = category_menu[item.item_name]["sizes"]
                if item.size not in valid_sizes:
                    errors.append(
                        f"{item.item_name}에 사용할 수 없는 사이즈다 (유효: {', '.join(valid_sizes)})"
                    )
        return errors
    
    def _calculate_total_price(self, items: List[OrderItem]) -> float:
        """총 가격을 계산한다"""
        total = 0.0
        for item in items:
            # 기본 가격
            item_total = item.price
            
            # 사이즈 추가 금액
            if item.size in SIZE_PRICES:
                item_total += SIZE_PRICES[item.size]
            
            # 옵션 추가 금액
            for option in item.options:
                if option in OPTION_PRICES:
                    item_total += OPTION_PRICES[option]
            
            # 수량 곱하기
            total += item_total * item.quantity
        
        return total
    
    def _check_combinations(self, items: List[OrderItem]) -> List[str]:
        """항목 조합에 대한 경고를 생성한다"""
        warnings = []
        
        # 카페인 과다 경고
        coffee_count = sum(
            item.quantity for item in items 
            if item.category == MenuCategory.COFFEE
        )
        if coffee_count >= 3:
            warnings.append(f"커피 {coffee_count}잔은 카페인이 많을 수 있다")
        
        # 디저트 대량 주문 경고
        dessert_count = sum(
            item.quantity for item in items 
            if item.category == MenuCategory.DESSERT
        )
        if dessert_count >= 4:
            warnings.append(f"디저트 {dessert_count}개는 많은 양이다")
        
        return warnings

# 검증 시스템 인스턴스 생성
validation_system = OrderValidationSystem()
print("주문 검증 시스템이 초기화되었다.")

주문 검증 시스템이 초기화되었다.


## 9. Recovery 메커니즘 구현

In [10]:
class OrderRecoverySystem:
    """주문 복구 메커니즘 시스템 클래스"""
    
    def __init__(self):
        self.recovery_model = "gpt-4o-mini"
    
    def suggest_alternatives(
        self,
        invalid_items: List[str],
        validation_errors: List[str]
    ) -> List[str]:
        """
        유효하지 않은 항목에 대한 대체 제안을 생성한다
        
        Args:
            invalid_items: 유효하지 않은 항목 리스트
            validation_errors: 검증 오류 메시지 리스트
        
        Returns:
            대체 제안 리스트
        """
        if not invalid_items and not validation_errors:
            return []
        
        suggestions = []
        
        # 재고 부족 항목 대체 제안
        for error in validation_errors:
            if "재고가 부족" in error:
                item_name = self._extract_item_name(error)
                alternatives = self._find_similar_items(item_name)
                if alternatives:
                    suggestions.append(
                        f"{item_name} 대신 {', '.join(alternatives)}를 추천한다"
                    )
        
        # 메뉴에 없는 항목 제안
        for item in invalid_items:
            similar = self._find_similar_items(item)
            if similar:
                suggestions.append(
                    f"{item}와 비슷한 메뉴: {', '.join(similar)}"
                )
        
        return suggestions
    
    def _extract_item_name(self, error_message: str) -> str:
        """오류 메시지에서 항목 이름을 추출한다"""
        # 간단한 추출 로직
        parts = error_message.split("의")
        if parts:
            return parts[0].strip()
        return ""
    
    def _find_similar_items(self, item_name: str) -> List[str]:
        """
        유사한 메뉴 항목을 찾는다
        
        Args:
            item_name: 찾을 항목 이름
        
        Returns:
            유사 항목 리스트
        """
        similar = []
        item_lower = item_name.lower()
        
        # 키워드 기반 유사도 검색
        for category, items in MENU_DATABASE.items():
            for menu_item in items.keys():
                menu_lower = menu_item.lower()
                
                # 부분 문자열 매칭
                if any(keyword in menu_lower for keyword in item_lower.split()):
                    similar.append(menu_item)
                elif any(keyword in item_lower for keyword in menu_lower.split()):
                    similar.append(menu_item)
        
        # 상위 3개만 반환
        return similar[:3]
    
    def generate_recovery_message(
        self,
        validation: OrderValidation,
        suggestions: List[str]
    ) -> str:
        """
        복구 메시지를 생성한다
        
        Args:
            validation: 검증 결과
            suggestions: 대체 제안 리스트
        
        Returns:
            복구 메시지
        """
        if validation.is_valid:
            return "주문이 정상적으로 처리되었다."
        
        message_parts = ["주문 처리 중 문제가 발견되었다:\n"]
        
        # 오류 메시지
        for error in validation.validation_errors:
            message_parts.append(f"  - {error}")
        
        # 대체 제안
        if suggestions:
            message_parts.append("\n다음 대안을 고려해 보라:")
            for suggestion in suggestions:
                message_parts.append(f"  - {suggestion}")
        
        return "\n".join(message_parts)

# 복구 시스템 인스턴스 생성
recovery_system = OrderRecoverySystem()
print("주문 복구 시스템이 초기화되었다.")

주문 복구 시스템이 초기화되었다.


## 10. Feedback 시스템 구현

In [11]:
class OrderFeedbackSystem:
    """주문 피드백 시스템 클래스"""
    
    def __init__(self):
        self.feedback_records = []
        self.category_success_rate = {
            "coffee": {"success": 0, "total": 0},
            "tea": {"success": 0, "total": 0},
            "dessert": {"success": 0, "total": 0},
            "side": {"success": 0, "total": 0}
        }
        self.common_errors = {}
    
    def record_order_result(
        self,
        parsed_order: ParsedOrder,
        validation: OrderValidation,
        classifications: List[CategoryClassification]
    ):
        """
        주문 처리 결과를 기록한다
        
        Args:
            parsed_order: 파싱된 주문
            validation: 검증 결과
            classifications: 카테고리 분류
        """
        record = {
            "timestamp": datetime.now(),
            "success": validation.is_valid,
            "total_items": parsed_order.total_items,
            "categories": [c.category.value for c in classifications],
            "errors": validation.validation_errors,
            "price": validation.total_price
        }
        self.feedback_records.append(record)
        
        # 카테고리별 성공률 업데이트
        for classification in classifications:
            cat = classification.category.value
            if cat in self.category_success_rate:
                self.category_success_rate[cat]["total"] += 1
                if validation.is_valid:
                    self.category_success_rate[cat]["success"] += 1
        
        # 오류 빈도 추적
        for error in validation.validation_errors:
            if error not in self.common_errors:
                self.common_errors[error] = 0
            self.common_errors[error] += 1
    
    def get_success_rate(self) -> float:
        """전체 주문 성공률을 계산한다"""
        if not self.feedback_records:
            return 0.0
        
        successful = sum(1 for r in self.feedback_records if r["success"])
        return successful / len(self.feedback_records)
    
    def get_category_performance(self) -> Dict[str, float]:
        """카테고리별 성공률을 반환한다"""
        performance = {}
        for category, stats in self.category_success_rate.items():
            if stats["total"] > 0:
                performance[category] = stats["success"] / stats["total"]
            else:
                performance[category] = 0.0
        return performance
    
    def get_most_common_errors(self, top_n: int = 5) -> List[tuple[str, int]]:
        """가장 흔한 오류를 반환한다"""
        sorted_errors = sorted(
            self.common_errors.items(),
            key=lambda x: x[1],
            reverse=True
        )
        return sorted_errors[:top_n]
    
    def generate_improvement_suggestions(self) -> List[str]:
        """
        성능 분석을 바탕으로 개선 제안을 생성한다
        
        Returns:
            개선 제안 리스트
        """
        if not self.feedback_records:
            return ["데이터가 충분하지 않다."]
        
        suggestions = []
        
        # 전체 성공률 확인
        success_rate = self.get_success_rate()
        if success_rate < 0.8:
            suggestions.append(
                f"전체 주문 성공률이 {success_rate:.1%}로 낮다. "
                f"주문 파싱 정확도를 개선하라."
            )
        
        # 카테고리별 성능 확인
        category_perf = self.get_category_performance()
        for category, rate in category_perf.items():
            if rate < 0.7:
                suggestions.append(
                    f"{category} 카테고리의 성공률이 {rate:.1%}로 낮다. "
                    f"해당 카테고리 처리 로직을 점검하라."
                )
        
        # 흔한 오류 분석
        common_errors = self.get_most_common_errors(3)
        if common_errors:
            error_msg = common_errors[0][0]
            error_count = common_errors[0][1]
            suggestions.append(
                f"'{error_msg}' 오류가 {error_count}회 발생했다. "
                f"이 문제를 우선적으로 해결하라."
            )
        
        # 재고 관리 제안
        stock_errors = [e for e in self.common_errors.keys() if "재고" in e]
        if stock_errors:
            suggestions.append(
                "재고 부족 오류가 자주 발생한다. 인기 메뉴의 재고를 늘려라."
            )
        
        return suggestions if suggestions else ["현재 시스템이 잘 작동하고 있다."]
    
    def get_statistics(self) -> Dict:
        """통계 정보를 반환한다"""
        if not self.feedback_records:
            return {"total_orders": 0}
        
        total_revenue = sum(r["price"] for r in self.feedback_records if r["success"])
        avg_items = sum(r["total_items"] for r in self.feedback_records) / len(self.feedback_records)
        
        return {
            "total_orders": len(self.feedback_records),
            "success_rate": self.get_success_rate(),
            "total_revenue": total_revenue,
            "avg_items_per_order": avg_items
        }

# 피드백 시스템 인스턴스 생성
feedback_system = OrderFeedbackSystem()
print("주문 피드백 시스템이 초기화되었다.")

주문 피드백 시스템이 초기화되었다.


## 11. Router 주문 응대 Classify Agent 통합 구현

In [12]:
class RouterOrderClassifyAgent:
    """Router 주문 응대 Classify Agent 메인 클래스"""
    
    def __init__(
        self,
        order_parser: OrderParser,
        category_router: CategoryRouter,
        validation_system: OrderValidationSystem,
        recovery_system: OrderRecoverySystem,
        memory: OrderMemory,
        feedback: OrderFeedbackSystem
    ):
        self.order_parser = order_parser
        self.category_router = category_router
        self.validation_system = validation_system
        self.recovery_system = recovery_system
        self.memory = memory
        self.feedback = feedback
        self.response_model = "gpt-4o-mini"
    
    def process_order(self, order_text: str) -> AgentResponse:
        """
        주문을 처리하고 최종 응답을 생성한다
        
        Args:
            order_text: 고객의 주문 텍스트
        
        Returns:
            AgentResponse 객체
        """
        print(f"\n{'='*60}")
        print(f"주문 접수: {order_text}")
        print(f"{'='*60}")
        
        # 1. 사용자 메시지를 메모리에 추가
        self.memory.add_message("user", order_text)
        
        # 2. 주문 파싱
        print("\n단계 1: 주문 파싱")
        context = self.memory.get_context()
        parsed_order = self.order_parser.parse_order(order_text, context)
        print(f"  - 파싱된 항목: {len(parsed_order.items)}개")
        for item in parsed_order.items:
            print(f"    * {item.item_name} x{item.quantity} ({item.category.value})")
        
        # 3. 카테고리별 분류 및 라우팅
        print("\n단계 2: 카테고리 분류 및 라우팅")
        classifications = self.category_router.classify_and_route(parsed_order)
        
        # 4. 주문 검증
        print("\n단계 3: 주문 검증")
        validation = self.validation_system.validate_order(parsed_order)
        print(f"  - 검증 결과: {'성공' if validation.is_valid else '실패'}")
        print(f"  - 총 가격: {validation.total_price:,.0f}원")
        
        if validation.validation_errors:
            print("  - 오류:")
            for error in validation.validation_errors:
                print(f"    * {error}")
        
        if validation.warnings:
            print("  - 경고:")
            for warning in validation.warnings:
                print(f"    * {warning}")
        
        # 5. Recovery 처리
        suggested_additions = []
        if not validation.is_valid:
            print("\n단계 4: Recovery 프로세스")
            suggestions = self.recovery_system.suggest_alternatives(
                validation.invalid_items,
                validation.validation_errors
            )
            suggested_additions = suggestions
            print("  - 대체 제안:")
            for suggestion in suggestions:
                print(f"    * {suggestion}")
        
        # 6. 고객 선호도 기반 추가 제안
        additional_suggestions = self._generate_upsell_suggestions(parsed_order)
        suggested_additions.extend(additional_suggestions)
        
        # 7. 응답 메시지 생성
        print("\n단계 5: 응답 메시지 생성")
        message = self._generate_response_message(
            parsed_order,
            validation,
            classifications,
            suggested_additions
        )
        
        # 8. 피드백 기록
        self.feedback.record_order_result(parsed_order, validation, classifications)
        
        # 9. 메모리에 응답 추가
        self.memory.add_message(
            "assistant",
            message,
            order_data=parsed_order if validation.is_valid else None
        )
        
        # 10. 확인이 필요한지 판단
        requires_confirmation = not validation.is_valid or len(validation.warnings) > 0
        
        # 11. 최종 응답 생성
        return AgentResponse(
            message=message,
            parsed_order=parsed_order,
            classifications=classifications,
            validation=validation,
            suggested_additions=suggested_additions,
            requires_confirmation=requires_confirmation
        )
    
    def _generate_upsell_suggestions(self, order: ParsedOrder) -> List[str]:
        """
        주문에 대한 추가 제안(업셀)을 생성한다
        
        Returns:
            추가 제안 리스트
        """
        suggestions = []
        
        # 커피를 주문했는데 디저트가 없는 경우
        has_coffee = any(item.category == MenuCategory.COFFEE for item in order.items)
        has_dessert = any(item.category == MenuCategory.DESSERT for item in order.items)
        
        if has_coffee and not has_dessert:
            suggestions.append("커피와 함께 즐길 디저트는 어떠한가? (쿠키, 스콘 추천)")
        
        # 음료만 주문한 경우
        all_beverages = all(
            item.category in [MenuCategory.COFFEE, MenuCategory.TEA] 
            for item in order.items
        )
        if all_beverages and order.total_items >= 2:
            suggestions.append("샌드위치나 베이글을 추가하면 든든하다")
        
        return suggestions
    
    def _generate_response_message(
        self,
        order: ParsedOrder,
        validation: OrderValidation,
        classifications: List[CategoryClassification],
        suggestions: List[str]
    ) -> str:
        """
        고객에게 전달할 응답 메시지를 생성한다
        
        Returns:
            응답 메시지
        """
        if validation.is_valid:
            # 성공적인 주문
            message_parts = ["주문이 접수되었다.\n"]
            message_parts.append("주문 내역:")
            
            for item in order.items:
                temp_str = f" ({item.temperature.value})" if item.temperature != Temperature.NONE else ""
                size_str = f" [{item.size}]" if item.size != "regular" else ""
                options_str = f" + {', '.join(item.options)}" if item.options else ""
                message_parts.append(
                    f"  - {item.item_name}{temp_str}{size_str}{options_str} x{item.quantity}"
                )
            
            message_parts.append(f"\n총 금액: {validation.total_price:,.0f}원")
            
            # 경고 메시지
            if validation.warnings:
                message_parts.append("\n안내 사항:")
                for warning in validation.warnings:
                    message_parts.append(f"  - {warning}")
            
            # 추가 제안
            if suggestions:
                message_parts.append("\n추가 제안:")
                for suggestion in suggestions:
                    message_parts.append(f"  - {suggestion}")
            
            return "\n".join(message_parts)
        
        else:
            # 실패한 주문
            return self.recovery_system.generate_recovery_message(validation, suggestions)

# Router 주문 응대 Classify Agent 인스턴스 생성
agent = RouterOrderClassifyAgent(
    order_parser=order_parser,
    category_router=category_router,
    validation_system=validation_system,
    recovery_system=recovery_system,
    memory=memory,
    feedback=feedback_system
)

print("\nRouter 주문 응대 Classify Agent가 초기화되었다.")


Router 주문 응대 Classify Agent가 초기화되었다.


## 12. 에이전트 테스트

In [13]:
# 테스트 주문 리스트
test_orders = [
    "아이스 아메리카노 2잔이랑 치즈케이크 하나 주세요",
    "핫 카페라떼 라지 사이즈에 샷 추가하고, 쿠키 3개 주세요",
    "녹차라떼 2개, 초코케이크 1개, 햄치즈샌드위치 2개 주문할게요",
    "바닐라라떼 1잔, 마카롱 5개 주세요. 포장해주세요",
]

# 각 주문에 대해 에이전트 실행
for i, order_text in enumerate(test_orders, 1):
    print(f"\n\n{'#'*60}")
    print(f"테스트 {i}/{len(test_orders)}")
    print(f"{'#'*60}")
    
    response = agent.process_order(order_text)
    
    print(f"\n{'='*60}")
    print(f"최종 응답")
    print(f"{'='*60}")
    print(response.message)
    print(f"\n확인 필요: {'예' if response.requires_confirmation else '아니오'}")



############################################################
테스트 1/4
############################################################

주문 접수: 아이스 아메리카노 2잔이랑 치즈케이크 하나 주세요

단계 1: 주문 파싱
  - 파싱된 항목: 2개
    * 아메리카노 x2 (coffee)
    * 치즈케이크 x1 (dessert)

단계 2: 카테고리 분류 및 라우팅

카테고리: coffee
  - 항목 수: 1개
  - 특별 처리: 불필요

카테고리: dessert
  - 항목 수: 1개
  - 특별 처리: 필요
  - 처리 사항: 케이크 포장 확인 필요

단계 3: 주문 검증
  - 검증 결과: 실패
  - 총 가격: 15,500원
  - 오류:
    * 치즈케이크에 사용할 수 없는 사이즈다 (유효: 1piece)

단계 4: Recovery 프로세스
  - 대체 제안:

단계 5: 응답 메시지 생성

최종 응답
주문 처리 중 문제가 발견되었다:

  - 치즈케이크에 사용할 수 없는 사이즈다 (유효: 1piece)

확인 필요: 예


############################################################
테스트 2/4
############################################################

주문 접수: 핫 카페라떼 라지 사이즈에 샷 추가하고, 쿠키 3개 주세요

단계 1: 주문 파싱
  - 파싱된 항목: 2개
    * 카페라떼 x1 (coffee)
    * 쿠키 x3 (dessert)

단계 2: 카테고리 분류 및 라우팅

카테고리: coffee
  - 항목 수: 1개
  - 특별 처리: 불필요

카테고리: dessert
  - 항목 수: 1개
  - 특별 처리: 불필요

단계 3: 주문 검증
  - 검증 결과: 실패
  - 총 가격: 14,500원
  - 오류:
    

## 13. 대화형 인터페이스

In [14]:
def order_interface():
    """대화형 주문 인터페이스 함수"""
    print("\n" + "="*60)
    print("커피 키오스크 Router 주문 응대 Classify Agent")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
    print("메뉴 보기는 'menu'를 입력하세요.")
    print("="*60 + "\n")
    
    while True:
        user_input = input("\n주문하실 내용: ").strip()
        
        if user_input.lower() in ['quit', 'exit', '종료']:
            print("\n주문을 종료한다. 이용해 주셔서 감사하다.")
            break
        
        if user_input.lower() == 'menu':
            print("\n=== 메뉴판 ===")
            for category, items in MENU_DATABASE.items():
                print(f"\n[{category.value.upper()}]")
                for name, info in items.items():
                    print(f"  {name}: {info['base_price']:,}원")
            continue
        
        if not user_input:
            continue
        
        response = agent.process_order(user_input)
        print(f"\n에이전트:\n{response.message}")

# 대화 시작 (주석을 제거하여 실행)
# order_interface()

## 14. 성능 분석 및 리포트

In [15]:
print("\n" + "="*60)
print("성능 분석 리포트")
print("="*60)

# 전체 통계
stats = feedback_system.get_statistics()
print("\n1. 전체 통계")
print(f"  - 총 주문 수: {stats.get('total_orders', 0)}건")
print(f"  - 성공률: {stats.get('success_rate', 0):.1%}")
print(f"  - 총 매출: {stats.get('total_revenue', 0):,.0f}원")
print(f"  - 주문당 평균 항목 수: {stats.get('avg_items_per_order', 0):.1f}개")

# 카테고리별 성능
print("\n2. 카테고리별 성능")
category_perf = feedback_system.get_category_performance()
for category, rate in category_perf.items():
    print(f"  - {category}: {rate:.1%}")

# 흔한 오류
print("\n3. 주요 오류 (Top 5)")
common_errors = feedback_system.get_most_common_errors(5)
if common_errors:
    for i, (error, count) in enumerate(common_errors, 1):
        print(f"  {i}. {error} ({count}회)")
else:
    print("  오류 없음")

# 개선 제안
print("\n4. 개선 제안")
suggestions = feedback_system.generate_improvement_suggestions()
for i, suggestion in enumerate(suggestions, 1):
    print(f"  {i}. {suggestion}")

# 고객 선호도
print("\n5. 고객 선호도")
favorites = memory.get_favorite_items(3)
if favorites:
    print(f"  - 인기 메뉴: {', '.join(favorites)}")
preferred_category = memory.get_preferred_category()
if preferred_category:
    print(f"  - 선호 카테고리: {preferred_category}")

# 메모리 상태
print("\n6. 메모리 상태")
print(f"  - {memory.get_summary()}")

print("\n" + "="*60)
print("튜토리얼 3: Router 주문 응대 Classify Agent 완료")
print("="*60)


성능 분석 리포트

1. 전체 통계
  - 총 주문 수: 4건
  - 성공률: 0.0%
  - 총 매출: 0원
  - 주문당 평균 항목 수: 4.5개

2. 카테고리별 성능
  - coffee: 0.0%
  - tea: 0.0%
  - dessert: 0.0%
  - side: 0.0%

3. 주요 오류 (Top 5)
  1. 치즈케이크에 사용할 수 없는 사이즈다 (유효: 1piece) (1회)
  2. 쿠키에 사용할 수 없는 사이즈다 (유효: 1piece) (1회)
  3. 녹차라떼의 온도 옵션이 지정되지 않았다 (1회)
  4. 녹차라떼에 사용할 수 없는 사이즈다 (유효: regular, large) (1회)
  5. 초코케이크에 사용할 수 없는 사이즈다 (유효: 1piece) (1회)

4. 개선 제안
  1. 전체 주문 성공률이 0.0%로 낮다. 주문 파싱 정확도를 개선하라.
  2. coffee 카테고리의 성공률이 0.0%로 낮다. 해당 카테고리 처리 로직을 점검하라.
  3. tea 카테고리의 성공률이 0.0%로 낮다. 해당 카테고리 처리 로직을 점검하라.
  4. dessert 카테고리의 성공률이 0.0%로 낮다. 해당 카테고리 처리 로직을 점검하라.
  5. side 카테고리의 성공률이 0.0%로 낮다. 해당 카테고리 처리 로직을 점검하라.
  6. '치즈케이크에 사용할 수 없는 사이즈다 (유효: 1piece)' 오류가 1회 발생했다. 이 문제를 우선적으로 해결하라.

5. 고객 선호도
  - 선호 카테고리: coffee

6. 메모리 상태
  - 총 주문: 0회, 선호 항목: 없음

튜토리얼 3: Router 주문 응대 Classify Agent 완료
